In [1]:
from keras.layers import Input, Lambda, Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

In [2]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                   brightness_range=[0.5, 1.5],  # Rango para ajustar el brillo 
                                   rotation_range=40,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory('Data/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_data = test_datagen.flow_from_directory('Data/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
     

Found 1200 images belonging to 6 classes.
Found 1200 images belonging to 6 classes.


In [3]:
# Ajuste de tamaño de imagenes a 224x224
IMAGE_SIZE = (224, 224)

train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'

# Se agrega el modelo preentrenado de VGG16
inception = VGG16(input_shape=IMAGE_SIZE + (3,), weights='imagenet', include_top=False)

In [4]:
inception.trainable = False

In [5]:
#Otra forma de conseguir el numero de clases que hay
folders = glob('Data/Test/*')

In [6]:
print(len(folders))

6


In [7]:
x = Flatten()(inception.output)
x = Dense(200, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.2)(x)
x = Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dense(25, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01))(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [8]:
modelo = Model(inputs=inception.input, outputs=prediction)
modelo.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
modelo.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [10]:
history = modelo.fit(train_data, validation_data=test_data, epochs=60, batch_size=15)

Epoch 1/60
38/38 [==============================] - 56s 1s/step - loss: 4.5325 - accuracy: 0.2092 - val_loss: 3.1511 - val_accuracy: 0.6108
Epoch 2/60
38/38 [==============================] - 22s 571ms/step - loss: 2.7266 - accuracy: 0.5975 - val_loss: 2.2175 - val_accuracy: 0.8200
Epoch 3/60
38/38 [==============================] - 22s 580ms/step - loss: 1.9594 - accuracy: 0.8667 - val_loss: 1.6144 - val_accuracy: 0.9417
Epoch 4/60
38/38 [==============================] - 22s 588ms/step - loss: 1.6123 - accuracy: 0.8708 - val_loss: 1.5158 - val_accuracy: 0.8208
Epoch 5/60
38/38 [==============================] - 23s 594ms/step - loss: 1.2607 - accuracy: 0.9542 - val_loss: 1.2529 - val_accuracy: 0.8325
Epoch 6/60
38/38 [==============================] - 23s 606ms/step - loss: 1.1664 - accuracy: 0.9242 - val_loss: 1.1929 - val_accuracy: 0.8433
Epoch 7/60
38/38 [==============================] - 24s 621ms/step - loss: 1.0494 - accuracy: 0.9467 - val_loss: 1.1755 - val_accuracy: 0.8408
Ep

In [11]:
modelo.save('Reconocimiento_Facial_Modelo.h5')

In [12]:
from sklearn.metrics import confusion_matrix
import numpy as np


predictions = modelo.predict(test_data)


true_labels = test_data.classes


predicted_labels = np.argmax(predictions, axis=1)


confusion_matrix = confusion_matrix(true_labels, predicted_labels)

print(confusion_matrix)

[[40 36 34 26 33 31]
 [41 31 49 21 23 35]
 [30 29 48 29 32 32]
 [31 40 45 23 28 33]
 [33 27 40 32 29 39]
 [29 36 40 37 27 31]]


In [13]:
true_labels = test_data.classes
predictions = modelo.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(true_labels, predicted_labels)


In [14]:
print(confusion_matrix)

[[26 33 49 34 24 34]
 [34 38 40 22 29 37]
 [43 29 42 27 30 29]
 [29 42 41 27 27 34]
 [30 32 40 29 34 35]
 [42 25 44 29 28 32]]
